# Загрузка Pandas и очистка данных

In [1125]:
import pandas as pd
import numpy as np
import re
import math

df = pd.read_csv('main_task.csv')

## Restaurant_id

In [1126]:
# Selecting number from ID
df['Restaurant_id'] = df['Restaurant_id'].apply(lambda x: re.findall(r'\d+', str(x))[0])

In [1127]:
# Changing the type of data in the column
df['Restaurant_id'] = df['Restaurant_id'].astype(int)

## Price Range

In [1128]:
# Замена значений Price Range на числовые:
dict_price_range = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
df['Price Range'] = df['Price Range'].replace(to_replace=dict_price_range)

In [1129]:
# Замена пустых значений Price Range на наиболее популярные значения:     
df['Price Range'] = df['Price Range'].\
fillna(df['Price Range'].value_counts().index[0])

## City

In [1130]:
# Создание Dummy-переменных
city = pd.get_dummies(df['City'], drop_first=True)

In [1131]:
# Конкатенация в один датафрейм
df = pd.concat([df, city], axis=1)

## Cuisine Style

In [1132]:
# Замена пустых значений на "Uknown":
df['Cuisine Style'] = df['Cuisine Style'].fillna("Uknown")

In [1133]:
# Применение функции форматирования значений столбца:
def list_cuisine(cell):
    """Функция форматирует значения в ячейках столбца"""
    cell = str(cell)
    cell = cell.replace("[", "")
    cell = cell.replace("]", "")
    cell = cell.replace("'", "")
    cell = cell.split(', ')
    return cell


df['Cuisine Style'] = df['Cuisine Style'].apply(list_cuisine)

In [1134]:
# TASK -- Какое среднее количество кухонь предлагается в одном ресторане?
round(df['Cuisine Style'].apply(lambda x: len(x)).mean(), 1)

2.6

In [1135]:
# TASK -- Сколько типов кухонь представлено в наборе данных (без "Uknown")?
df = df.explode('Cuisine Style')
len(df['Cuisine Style'].value_counts())-1

125

In [1136]:
# TASK -- Какая кухня представлена в наибольшем количестве ресторанов?
df = df.explode('Cuisine Style')
df['Cuisine Style'].value_counts().index[0]

'Vegetarian Friendly'

In [1137]:
# Создание Dummy-переменных:
cuisine = pd.get_dummies(df['Cuisine Style'], drop_first=True)

In [1138]:
# Конкатенация в один датафрейм
df = pd.concat([df, cuisine], axis=1)

## Reviews

In [1139]:
# Remove empty cells
df = df[df['Reviews'] != '[[], []]']

In [1140]:
# Find only data's numbers
df['Reviews'] = df['Reviews'].apply(lambda x: re.findall(r'\d+/\d+/\d+', str(x)))

In [1141]:
# TASK -- The latest review
# df = df.explode('Reviews')
# df['Reviews'] = pd.to_datetime(df['Reviews'])
# df['Reviews'].max()

In [1142]:
# New columns for 1st and 2nd review
df['1-Review'] = df['Reviews'].apply(lambda x: x[0])
df['2-Review'] = df['Reviews'].apply(lambda x: x[-1])

In [1143]:
# Conversion to date
df['1-Review'] = pd.to_datetime(df['1-Review'])
df['2-Review'] = pd.to_datetime(df['2-Review'])

In [1144]:
# Number of dates between reviews
df['Delta-Review'] = (abs(df['2-Review'] - df['1-Review'])).dt.days

In [1145]:
# TASK -- Max number of dates between reviews
df['Delta-Review'].sort_values(ascending=False).iloc[0]

3207

## Number of Reviews

In [1146]:
# Filling in the gaps with average number of reviews
df['Number of Reviews'] = df['Number of Reviews'].fillna(round(df['Number of Reviews'].mean()))

# Changing the type of data in the column
df['Number of Reviews'] = df['Number of Reviews'].astype(int)

## Rest Unused Columns

In [1147]:
# Deletion
df = df.drop(['City', 'Cuisine Style', 'Price Range', 'Reviews',\
              'URL_TA', 'ID_TA', '1-Review', '2-Review'], axis=1)

## Feature Engineering

In [1148]:
df['Rating/Ranking'] = df['Rating'] / df['Ranking']

## Normalization

In [1149]:
df['Ranking'] = np.log(df['Ranking'])
df['Rating'] = np.log(df['Rating'])
df['Number of Reviews'] = np.log(df['Number of Reviews'])
df['Rating/Ranking'] = np.log(df['Rating/Ranking'])

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [1150]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [1151]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [1152]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [1153]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [1154]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [1155]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.0021766003590285407
